#### Characteristic vibrational energy relaxation time

The characteristic vibrational energy relaxation time is defined as the time required by the vibrational energy mode to reach the 63.21% of its equilibrium value
\begin{equation}
    \tau=t\hspace{3mm}\text{s.t.}\hspace{3mm}\epsilon_{int}\left(t\right) = 0.6321\epsilon^*_{int}
\end{equation}
The Millikan-White formula is
\begin{equation}
    \tau=\frac{1}{p}\exp\left[a\left(T^{-1/3}-b\right)-c\right]
\end{equation}
with $c=18.42$.

In [ ]:
path_to_ronek = "/home/zanardi/Codes/ML/RONEK/ronek/"

import sys
import importlib
if (importlib.util.find_spec("ronek") is None):
  sys.path.append(path_to_ronek)

In [ ]:
env_cfg = {
  "backend": "torch",
  "device": "cuda",
  "device_idx": 0,
  "nb_threads": 8,
  "epsilon": 1e-10,
  "floatx": "float64"
}

from ronek import env
env.set(**env_cfg)

In [ ]:
import torch
import numpy as np
import scipy as sp
import pandas as pd

from scipy import constants
from ronek import backend as bkd
from ronek import systems as sys_mod

Inputs

In [ ]:
T = 1e4
lev_file = "/home/zanardi/Codes/ML/RONEK/ronek/examples/RVC_O3/database/O2.csv"
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/RVC_O3/database/"

Initialize thermochemical system

In [ ]:
system = sys_mod.TASystem(
  species={
    k: path_to_dtb + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  rates_coeff=path_to_dtb + "/kinetics.hdf5",
  use_einsum=False,
  use_factorial=True,
  use_arrhenius=True
)
system.update_fom_ops(T)

In [ ]:
levels = pd.read_csv(lev_file)
gi = levels["g"].values
ei = levels["E"].values
ei -= ei.min()

> Remove dissociation processes

In [ ]:
for k in ("fwd", "bwd"):
  system.kin.rates["m-a"]["d"][k][::] = 0.0

> Compute eigendecomposition of the excitation processes matrix

In [ ]:
k = system._update_fom_ops(system.kin.rates)["ed"]
l, v = sp.linalg.eig(k)
vinv = sp.linalg.inv(v)
l, v, vinv = [bkd.to_torch(x) for x in (l, v, vinv)]

Solve the system for different pressure values
> Time grid

In [ ]:
t = system.get_tgrid(1e-12, 1e-3, num=200)
t = bkd.to_torch(t)

> Initial conditions

In [ ]:
p0 = np.geomspace(1e3,1e4,10)
T0 = 500.0
x0_a = 0.05

> Solve

In [ ]:
def solve(p0):
  n0 = system.mix.get_init_sol(T0, x0_a, p0, mu_type="mole")
  n0 = bkd.to_torch(n0)
  n = []
  for ti in t:
    li = torch.exp(ti*l*n0[0])
    ni = v @ (li * (vinv @ n0[1:]))
    n.append(bkd.to_numpy(ni))
  n = np.vstack(n).T
  return n / np.sum(n, axis=0)

In [ ]:
f = [solve(p0_i) for p0_i in p0]

Compute vibrational energy relaxation time

In [ ]:
i_vib = (levels["jqn"] == 0)

In [ ]:
def compute_tau(p0, fi, ei):
  e_vib = np.sum(fi[i_vib].T * ei[i_vib], axis=1)
  e_tau = 0.6321 * e_vib[-1]
  index = np.argmin(np.absolute(e_vib-e_tau))
  tau_p = float(t[index])*p0/constants.atm
  return tau_p

In [ ]:
tau_p = np.array([compute_tau(p0[i], f[i], ei) for i in range(len(p0))])

Compute coefficients for the Millikan-White formula

In [ ]:
def loss_mw_formula(x, tau_p, c):
  a, b = x
  lhs = np.log(tau_p)
  rhs = a*(np.power(T, -1/3)-b) - c
  res = lhs - rhs
  return np.sum(res**2)

In [ ]:
c = 18.42
x0 = np.array([40.0, 1.0])
a, b = sp.optimize.minimize(loss_mw_formula, x0, args=(tau_p, c), method="CG").x

In [ ]:
text = f"- The MW coefficients are [a, b, c]: {a}, {b}, {c}"
with open("./tau.txt", "w") as file:
  file.write(text)
print(text)